In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import OrderedDict
from torch.nn import init
import numpy as np

def conv3x3(in_channels, out_channels, stride=1, 
            padding=1, bias=True, groups=1):    
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=3,
        stride=stride,
        padding=padding,
        bias=bias,
        groups=groups)

def upconv2x2(in_channels, out_channels, mode='transpose'):
    if mode == 'transpose':
        return nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size=2,
            stride=2)
    else:
        # out_channels is always going to be the same
        # as in_channels
        return nn.Sequential(
            nn.Upsample(mode='bilinear', scale_factor=2),
            conv1x1(in_channels, out_channels))

def conv1x1(in_channels, out_channels, groups=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=1,
        groups=groups,
        stride=1)


class DownConv(nn.Module):
    """
    A helper Module that performs 2 convolutions and 1 MaxPool.
    A ReLU activation follows each convolution.
    """
    def __init__(self, in_channels, out_channels, pooling=True):
        super(DownConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.pooling = pooling

        self.conv1 = conv3x3(self.in_channels, self.out_channels)
        self.conv2 = conv3x3(self.out_channels, self.out_channels)

        if self.pooling:
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        before_pool = x
        if self.pooling:
            x = self.pool(x)
        return x, before_pool


class UpConv(nn.Module):
    """
    A helper Module that performs 2 convolutions and 1 UpConvolution.
    A ReLU activation follows each convolution.
    """
    def __init__(self, in_channels, out_channels, 
                 merge_mode='concat', up_mode='transpose'):
        super(UpConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.merge_mode = merge_mode
        self.up_mode = up_mode

        self.upconv = upconv2x2(self.in_channels, self.out_channels, 
            mode=self.up_mode)

        if self.merge_mode == 'concat':
            self.conv1 = conv3x3(
                2*self.out_channels, self.out_channels)
        else:
            # num of input channels to conv2 is same
            self.conv1 = conv3x3(self.out_channels, self.out_channels)
        self.conv2 = conv3x3(self.out_channels, self.out_channels)

    def forward(self, from_down, from_up):
        """ Forward pass
        Arguments:
            from_down: tensor from the encoder pathway
            from_up: upconv'd tensor from the decoder pathway
        """
        from_up = self.upconv(from_up)
        if self.merge_mode == 'concat':
            x = torch.cat((from_up, from_down), 1)
        else:
            x = from_up + from_down
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x


class UNet(nn.Module):
    """ `UNet` class is based on https://arxiv.org/abs/1505.04597

    The U-Net is a convolutional encoder-decoder neural network.
    Contextual spatial information (from the decoding,
    expansive pathway) about an input tensor is merged with
    information representing the localization of details
    (from the encoding, compressive pathway).

    Modifications to the original paper:
    (1) padding is used in 3x3 convolutions to prevent loss
        of border pixels
    (2) merging outputs does not require cropping due to (1)
    (3) residual connections can be used by specifying
        UNet(merge_mode='add')
    (4) if non-parametric upsampling is used in the decoder
        pathway (specified by upmode='upsample'), then an
        additional 1x1 2d convolution occurs after upsampling
        to reduce channel dimensionality by a factor of 2.
        This channel halving happens with the convolution in
        the tranpose convolution (specified by upmode='transpose')
    """

    def __init__(self, num_classes, in_channels=3, depth=5, 
                 start_filts=64, up_mode='transpose', 
                 merge_mode='concat'):
        """
        Arguments:
            in_channels: int, number of channels in the input tensor.
                Default is 3 for RGB images.
            depth: int, number of MaxPools in the U-Net.
            start_filts: int, number of convolutional filters for the 
                first conv.
            up_mode: string, type of upconvolution. Choices: 'transpose'
                for transpose convolution or 'upsample' for nearest neighbour
                upsampling.
        """
        super(UNet, self).__init__()

        if up_mode in ('transpose', 'upsample'):
            self.up_mode = up_mode
        else:
            raise ValueError("\"{}\" is not a valid mode for "
                             "upsampling. Only \"transpose\" and "
                             "\"upsample\" are allowed.".format(up_mode))
    
        if merge_mode in ('concat', 'add'):
            self.merge_mode = merge_mode
        else:
            raise ValueError("\"{}\" is not a valid mode for"
                             "merging up and down paths. "
                             "Only \"concat\" and "
                             "\"add\" are allowed.".format(up_mode))

        # NOTE: up_mode 'upsample' is incompatible with merge_mode 'add'
        if self.up_mode == 'upsample' and self.merge_mode == 'add':
            raise ValueError("up_mode \"upsample\" is incompatible "
                             "with merge_mode \"add\" at the moment "
                             "because it doesn't make sense to use "
                             "nearest neighbour to reduce "
                             "depth channels (by half).")

        self.num_classes = num_classes
        self.in_channels = in_channels
        self.start_filts = start_filts
        self.depth = depth
        self.down_convs = []
        self.up_convs = []

        # create the encoder pathway and add to a list
        for i in range(depth):
            ins = self.in_channels if i == 0 else outs
            outs = self.start_filts*(2**i)
            pooling = True if i < depth-1 else False

            down_conv = DownConv(ins, outs, pooling=pooling)
            self.down_convs.append(down_conv)

        # create the decoder pathway and add to a list
        # - careful! decoding only requires depth-1 blocks
        for i in range(depth-1):
            ins = outs
            outs = ins // 2
            up_conv = UpConv(ins, outs, up_mode=up_mode,
                merge_mode=merge_mode)
            self.up_convs.append(up_conv)

        self.conv_final = conv1x1(outs, self.num_classes)

        # add the list of modules to current module
        self.down_convs = nn.ModuleList(self.down_convs)
        self.up_convs = nn.ModuleList(self.up_convs)

        self.reset_params()

    @staticmethod
    def weight_init(m):
        if isinstance(m, nn.Conv2d):
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0)


    def reset_params(self):
        for i, m in enumerate(self.modules()):
            self.weight_init(m)


    def forward(self, x):
        encoder_outs = []
         
        # encoder pathway, save outputs for merging
        for i, module in enumerate(self.down_convs):
            x, before_pool = module(x)
            encoder_outs.append(before_pool)
        
        for i, module in enumerate(self.up_convs):
            before_pool = encoder_outs[-(i+2)]
            x = module(before_pool, x)
        
        # No softmax is used. This means you need to use
        # nn.CrossEntropyLoss is your training script,
        # as this module includes a softmax already.
        x = self.conv_final(x)
        return x




In [2]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
import random
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets.utils as utils
import torch.nn.functional as F

#Weigth coef for the Unsupervised
def dice_loss(logits, targets, is_modelT_preds=True): 
  if is_modelT_preds:
    targets_animal = torch.unsqueeze(targets, dim=1)
  targets_animal = torch.squeeze(targets).to(torch.float32)
  preds_animal = F.softmax(logits, dim=1)
  preds_animal = preds_animal[:,1,:,:].to(torch.float32)
  eps = 1e-6
  intersection = (preds_animal * targets_animal).sum()
  dice_coef = (2. * intersection + eps) / ((preds_animal**2).sum() + (targets_animal**2).sum() + eps)
  dice_loss = 1 - dice_coef
  return dice_loss

@torch.no_grad()
def wt(rampup_length, current, alpha, wait_period = 5):

  if current < wait_period:
    return 0.0
    
  else:
    if rampup_length == 0:
                return 1.0
    else:
        current -= wait_period
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(alpha * np.exp(-5.0 * phase * phase))


#update the Teacher weight
@torch.no_grad()
def update_ema_variables(model, ema_model, alpha, global_step): 
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)


@torch.no_grad()
def evaluate_model(model, dataloader):
  
  model.eval()
  intersection_total, union_total = 0, 0
  pixel_correct, pixel_count = 0, 0
    
  for data in dataloader:
    imgs, labels = data
    imgs, labels = imgs.cuda(), labels.cuda()
    logits = model(imgs)
    preds = torch.argmax(logits, dim=1)
    targets = torch.squeeze(labels)
            
    intersection_total += torch.logical_and(preds, targets).sum()
    union_total += torch.logical_or(preds, targets).sum()
            
    pixel_correct += (preds == targets).sum()
    pixel_count += targets.numel()

  iou = (intersection_total / union_total).item()
  accuracy = (pixel_correct / pixel_count).item()
  
  return accuracy, iou




In [3]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
import random
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets.utils as utils

random.seed(200)  # Fix randomness

# mean and std of whole image dataset
DATA_MEAN = torch.asarray([0.4803, 0.4497, 0.3960])
DATA_STD = torch.asarray([0.2665, 0.2623, 0.2707])


# Each pixel in a mask image can take one of three values: 1, 2, or 3. 1 means that this pixel of an image belongs to the class pet, 2 - to the class background, 3 - to the class border.
def preprocess_mask(mask):

    mask[mask == (2.0 / 255)] = 0.0
    mask[(mask == 1.0 / 255) | (mask == 3.0 / 255)] = 1.0
    return mask


class OxfordPetDataset_with_labels(Dataset):
    def __init__(self, images_filenames, images_directory, masks_directory, transform_data_1=None, transform_mask_1=None, transform_2=None):
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.masks_directory = masks_directory

        self.transform_data_1 = transform_data_1
        self.transform_mask_1 = transform_mask_1
        self.transform_2 = transform_2

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]
        image = Image.open(os.path.join(self.images_directory, image_filename))
        mask = Image.open(
            os.path.join(self.masks_directory,
                         image_filename.replace(".jpg", ".png")),
        )

        if self.transform_data_1 is not None:
            image = self.transform_data_1(image)

        if self.transform_mask_1 is not None:
            mask = self.transform_mask_1(mask)

        mask = preprocess_mask(mask)

        if self.transform_2 is not None:
            mask = self.transform_2(mask)
            image = self.transform_2(image)

        mask[mask <= 0.5] = 0
        mask[mask > 0.5] = 1

        return image, mask

def readable_images(images_filenames, images_directory):
    """
    Remove the data that are not readable
    """
    correct_images_filenames = []

    for i in images_filenames:
        try:
            Image.open(os.path.join(images_directory, i))
            correct_images_filenames.append(i)
        except:
            continue

    return correct_images_filenames


def are_images_all_RGB(images_filenames, images_directory):
    """
    Remove the data that do not have shape of (3,_,_)
    """
    correct_images_filenames = []
    transform = transforms.Compose([transforms.ToTensor()])
    for i in images_filenames:
        img = Image.open(os.path.join(images_directory, i))
        img = transform(img)
        if img.shape[0] == 3:
            correct_images_filenames.append(i)
    return correct_images_filenames

def get_supervised_data(percentage_labelled, percentage_validation, percentage_test, img_resize=64):
    random.seed(200)
    images_directory = os.path.join("/kaggle/input/oxford-pets/images/images")
    masks_directory = os.path.join("/kaggle/input/oxford-pets/annotations/annotations/trimaps")
    images_filenames = list(sorted(os.listdir(images_directory)))
    
    correct_images_filenames = readable_images(
        images_filenames, images_directory)
    correct_images_filenames = are_images_all_RGB(
        correct_images_filenames, images_directory)

    random.shuffle(correct_images_filenames)

    nb_data = len(correct_images_filenames)
    transform_data_1 = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize(DATA_MEAN, DATA_STD)
         ])

    transform_mask_1 = transforms.Compose(
        [transforms.ToTensor()])

    transform_2 = transforms.Compose(
        [transforms.Resize((img_resize, img_resize))]
    )

    train_images_filenames = correct_images_filenames[0:int(
        nb_data*percentage_labelled)]
    validation_images_filenames = correct_images_filenames[int(
        nb_data*(1-percentage_validation - percentage_test)):int(nb_data*(1 - percentage_test))]
    test_images_filenames = correct_images_filenames[int(
        nb_data*(1 - percentage_test)):]

    train_data = OxfordPetDataset_with_labels(train_images_filenames,
                                              images_directory, masks_directory, transform_data_1, transform_mask_1, transform_2)

    validation_data = OxfordPetDataset_with_labels(
        validation_images_filenames, images_directory, masks_directory, transform_data_1, transform_mask_1, transform_2)

    # test data
    test_data = OxfordPetDataset_with_labels(
        test_images_filenames, images_directory, masks_directory, transform_data_1, transform_mask_1, transform_2)
    print(f'Loaded {len(images_filenames)} images')

    return train_data, validation_data, test_data
# (Keep this commented)
# Example  : mixed_train_loader, val_loader, test_loader = get_data(0.2, 0.8, 0.2, 0.2)


In [4]:
'''
https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
'''
import numpy as np
from torchvision.utils import save_image
import torch
import torchvision
from random import choice
from torchvision.transforms import RandomInvert, RandomRotation




def augmentation(batch, masks, var):
    augmented_batch = batch + torch.empty(batch.shape).normal_(mean=0,std=var)
    augmented_batch = torch.clip(augmented_batch, 0, 1)
    inversion = RandomInvert(np.random.lognormal(1,var))
    augmented_batch = inversion(augmented_batch)
    augmented_batch = torchvision.transforms.functional.adjust_saturation(augmented_batch, np.random.lognormal(1, var))
    rotater = RandomRotation(degrees=(-var*90,var*90))
    augmented_batch = rotater(batch)
    augmented_mask = rotater(masks)
    return augmented_batch.type(torch.float32), augmented_mask.type(torch.float32)

In [14]:
def train(config):
    model = UNet(2, in_channels = 3, depth=config['depth'])
    model = nn.DataParallel(model)
    model.cuda()
    losses, accsTr, IousTr, accsVal, IousVal = [], [], [], [], []
    optimizer = Adam(model.parameters(), lr=config['lr'])
    loss_fn = dice_loss
    s = "/kaggle/working/Models/" + f"{config['batch_size']}-{config['gaussian_noise']}-{config['lr']}-{config['lr_decay']}-{config['depth']}/"
    if config['lr_decay'] is not None:
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['lr_decay'])
    else:
        scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    num_workers = 1

    train_loader = DataLoader(
        train_data,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers = num_workers
    )
    train_loader_eval = DataLoader(
        train_data,
        batch_size=128,
        shuffle=False,
        num_workers = num_workers
    )
    val_loader = DataLoader(
        val_data,
        batch_size=128,
        shuffle=False,
        num_workers = num_workers
    )


    for epoch in range(max_epochs):
        model.train()
        running_loss = 0
        
        for step, data in enumerate(train_loader):
            optimizer.zero_grad()
            imgs, labels = data
            
            imgs_aug, labels_aug = augmentation(imgs, labels, config['gaussian_noise'])
            imgs_aug, labels_aug = imgs_aug.cuda(), labels_aug.cuda()
            output = model.forward(imgs_aug)

            loss = loss_fn(output, labels_aug)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        losses.append(running_loss)

        accTr, IouTr = evaluate_model(model, train_loader_eval)  
        accVal, IouVal = evaluate_model(model, val_loader)   

        accsTr.append(accTr)
        IousTr.append(IouTr)
        accsVal.append(accVal)
        IousVal.append(IouVal)
        
        
        
        df = pd.DataFrame({'loss': losses, 'accTr': accsTr, 'IouTr': IousTr, 'accVal': accsVal, 'IouVal': IousVal})
        df.to_csv(s+f"Results.csv")

        session.report({'validation_iou': IouVal, 'running_loss': running_loss, 'train_iou': IouTr, 'validation_accuracy': accVal, 'train_accuracy': accTr})

    
def main(config):
    
    hyperopt_search = HyperOptSearch(
        metric="validation_iou", mode="max"
    )
    hyperopt_search = tune.search.ConcurrencyLimiter(hyperopt_search, max_concurrent=4)
    resources_per_trial = {"cpu": .5, "gpu": .5}
    stopper = tune.stopper.TrialPlateauStopper(
        metric = 'validation_iou', 
        std = 0.01, 
        num_results = 3, 
        grace_period = 1,
        mode = 'min',
        metric_threshold=0.7
    )
    tuner = tune.Tuner(
        tune.with_resources(train, resources=resources_per_trial),
        tune_config=tune.TuneConfig(
            search_alg=hyperopt_search,
            num_samples=50
        ),
        param_space=config,
        run_config=air.RunConfig(stop=stopper)
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("validation_iou", "max")
    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation IoU: {}".format(
        best_result.metrics["validation_iou"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["validation_accuracy"]))
    print("Best trial final train IoU: {}".format(
        best_result.metrics["train_iou"]))
    print("Best trial final train accuracy: {}".format(
        best_result.metrics["train_accuracy"]))
    print("Best running loss: {}".format(
        best_result.metrics["running_loss"]))
    
    return best_result


In [6]:
!mkdir Models

In [7]:
from torch.utils.data import DataLoader
from ray.tune.search.hyperopt import HyperOptSearch
from torch.optim import Adam, lr_scheduler
import torch
import ray
from ray import tune, air
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
import pandas as pd
import os

# Do NOT touch these hyperparameters
img_resize = 64
percentage_validation = 0.1
percentage_test = 0.1

# Change percentage labelled to match the current optimisation parameters.
percentage_labelled = 0.25
max_epochs = 20
#Training params
HYPERPARAMETER_SPACE = {
    'batch_size': tune.choice([16, 32, 64]),
    'gaussian_noise': tune.choice([0.01, 0.1, 1, 10]),
    'lr': tune.choice([1e-2, 1e-3, 1e-4, 1e-5]),
    'lr_decay': tune.choice([0.9, 0.99, 1, None]),
    'depth': tune.choice([3,4,5]),
}

for batch_size in [16, 32, 64]:
    for gaussian_noise in [0.01, 0.1, 1, 10]:
        for lr in [1e-2, 1e-3, 1e-4, 1e-5]:
            for lr_decay in [0.9, 0.99, 1, None]:
                for depth in [3,4,5]:
                    os.system(f"mkdir Models/{batch_size}-{gaussian_noise}-{lr}-{lr_decay}-{depth}")
CURRENT_BEST_PARAMS = [{
    'batch_size': 32,
    'gaussian_noise': 1,
    'lr': 1e-3,
    'lr_decay': 0.99,
    'depth': 3
}]




In [8]:
train_data, val_data, test_data = get_supervised_data(percentage_labelled, percentage_validation, percentage_test, img_resize=64)


Loaded 7393 images


In [ ]:
main(HYPERPARAMETER_SPACE)


(train pid=6618) E0406 09:45:42.261101795    6675 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
(train pid=6487) E0406 09:46:43.021408058    6528 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,running_loss,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_accuracy,train_iou,training_iteration,trial_id,validation_accuracy,validation_iou,warmup_time
train_0020b427,2023-04-06_09-55-27,True,,8c7e7ced407f433c87d107ba119a6a3f,,fe5c90fb383f,3,172.19.2.2,6711,24.4367,578.304,197.155,578.304,1680774927,0,,0.411811,0.411811,3,0020b427,0.417831,0.417831,0.0252821
train_01af0d6b,2023-04-06_13-00-54,True,,12135f0a5d7e4893888e1b9905b85942,"26_batch_size=32,depth=4,gaussian_noise=0.0100,lr=0.0010,lr_decay=0.9000",fe5c90fb383f,20,172.19.2.2,19909,4.08156,3914.56,192.965,3914.56,1680786054,0,,0.92847,0.843033,20,01af0d6b,0.887408,0.768834,0.0315158
train_0e5291e2,2023-04-06_13-31-58,True,,0abd72ff1774442985e246729c24ea68,"31_batch_size=16,depth=5,gaussian_noise=0.0100,lr=0.0010,lr_decay=0.9000",fe5c90fb383f,20,172.19.2.2,23068,6.81555,3921.56,200.99,3921.56,1680787918,0,,0.939098,0.863588,20,0e5291e2,0.893951,0.776969,0.0183592
train_1c59b1db,2023-04-06_10-27-22,True,,f791ce0806f1470eb38b6ecf520a5e62,,fe5c90fb383f,5,172.19.2.2,9409,26.9717,958.916,189.324,958.916,1680776842,0,,0.753607,0.577565,5,1c59b1db,0.752983,0.578517,0.0129607
train_1e700e63,2023-04-06_10-28-00,True,,5f0aa936fe5a43378d97cd02540b3683,,fe5c90fb383f,3,172.19.2.2,10136,9.99357,577.368,191.753,577.368,1680776880,0,,0.411929,0.411858,3,1e700e63,0.417949,0.41788,0.016789
train_293d20c3,2023-04-06_12-05-52,True,,2822163c1f7f459981a37a0b0d1b9be5,"21_batch_size=16,depth=4,gaussian_noise=0.0100,lr=0.0001,lr_decay=0.9000",fe5c90fb383f,20,172.19.2.2,14669,7.21534,3882.86,195.266,3882.86,1680782752,0,,0.931574,0.848741,20,293d20c3,0.88304,0.759462,0.0160482
train_2dbc1b70,2023-04-06_11-07-20,True,,83f07480c31e4fdda6325f61874cdb20,,fe5c90fb383f,3,172.19.2.2,14147,14.3165,595.875,191.331,595.875,1680779240,0,,0.734074,0.532663,3,2dbc1b70,0.734327,0.535658,0.0331132
train_36ba8f37,2023-04-06_10-47-25,True,,2b1dd675d5be4bc79229d773d3f214ab,,fe5c90fb383f,3,172.19.2.2,12101,9.76775,591.568,195.314,591.568,1680778045,0,,0.41184,0.411822,3,36ba8f37,0.417865,0.417845,0.00829268
train_390cf85c,2023-04-06_10-11-09,True,,a1e7ecfe08934371b02d76d9813200a1,,fe5c90fb383f,5,172.19.2.2,7597,14.675,974.249,189.664,974.249,1680775869,0,,0.726241,0.528426,5,390cf85c,0.726956,0.531242,0.00724649
train_3c33d87c,2023-04-06_11-09-38,True,,d47c2fc4b27747f482d279be99bf4513,,fe5c90fb383f,3,172.19.2.2,14413,12.483,594.575,197.332,594.575,1680779378,0,,0.411811,0.411811,3,3c33d87c,0.417831,0.417831,0.0108631


(train pid=6529) E0406 09:51:50.165680686    6587 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
(train pid=6487) E0406 09:52:55.973460430    6528 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
(train pid=7597) E0406 09:55:01.310143592    7654 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
(train pid=7876) E0406 09:55:54.116057569    7935 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
(train pid=7597) E0406 09:57:55.182848627    7654 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
(train pid=7704) E0406 10:00:03.867253715    7747 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
(train pid=7783) E0406 10:00:18.499803482    7843 fork_posix.cc:76]           Other threads are curr

In [10]:
!ls

E0406 09:09:06.233536414      23 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers


Models	__notebook_source__.ipynb
